In [1]:
import pandas as pd
import numpy as np 
import scipy as sc
from pandas_profiling import ProfileReport

In [2]:
pva_data = pd.read_csv('donors.csv')

C:\Users\Frederico Rodrigues\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (9,412) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
pva_data.head()

,Unnamed: 0,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,...,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,0,2009-01-01,GRI,0,IL,61081,,,1957-12-01,0,...,7.741935,95515,0,L,4,E,X,X,X,C
1,1,2014-01-01,BOA,1,CA,91326,,,1972-02-01,0,...,15.666667,148535,0,L,2,G,X,X,X,A
2,2,2010-01-01,AMH,1,NC,27017,,,NaN,0,...,7.481481,15078,1,L,4,E,X,X,X,C
3,3,2007-01-01,BRY,0,CA,95953,,,1948-01-01,0,...,6.812500,172556,1,L,4,E,X,X,X,C
4,4,2006-01-01,,0,FL,33176,,,1940-01-01,0,...,6.864865,7112,1,L,2,F,X,X,X,A


In [4]:
pva_data.dtypes

Unnamed: 0     int64
ODATEDW       object
OSOURCE       object
TCODE          int64
STATE         object
               ...  
RFA_2A        object
MDMAUD_R      object
MDMAUD_F      object
MDMAUD_A      object
GEOCODE2      object
Length: 476, dtype: object

In [5]:
# duplicated observations
pva_data.duplicated().sum()

0

In [6]:
pva_data.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,95412.0,47705.500000,27543.216279,0.000000,23852.750000,47705.500000,71558.250000,95411.0
TCODE,95412.0,54.223117,953.844476,0.000000,0.000000,1.000000,2.000000,72002.0
NUMCHLD,12386.0,1.527773,0.806861,1.000000,1.000000,1.000000,2.000000,7.0
INCOME,74126.0,3.886248,1.854960,1.000000,2.000000,4.000000,5.000000,7.0
WEALTH1,50680.0,5.345699,2.742490,0.000000,3.000000,6.000000,8.000000,9.0
...,...,...,...,...,...,...,...,...
TIMELAG,85439.0,8.093739,8.213242,0.000000,4.000000,6.000000,11.000000,1088.0
AVGGIFT,95412.0,13.347786,10.769997,1.285714,8.384615,11.636364,15.477955,1000.0
CONTROLN,95412.0,95778.176959,55284.596094,1.000000,47910.750000,95681.500000,143643.500000,191779.0
HPHONE_D,95412.0,0.500618,0.500002,0.000000,0.000000,1.000000,1.000000,1.0


In [7]:
pva_data.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Unnamed: 0,95412,NaN,NaN,NaN,47705.5,27543.2,0,23852.8,47705.5,71558.2,95411
ODATEDW,95412,54,2015-01-01,15358,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OSOURCE,95412,896,MBC,4539,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCODE,95412,NaN,NaN,NaN,54.2231,953.844,0,0,1,2,72002
STATE,95412,57,CA,17343,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
RFA_2A,95412,4,F,46964,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MDMAUD_R,95412,5,X,95118,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MDMAUD_F,95412,4,X,95118,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MDMAUD_A,95412,5,X,95118,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
pva_data.columns

Index(['Unnamed: 0', 'ODATEDW', 'OSOURCE', 'TCODE', 'STATE', 'ZIP', 'MAILCODE',
       'PVASTATE', 'DOB', 'NOEXCH',
       ...
       'AVGGIFT', 'CONTROLN', 'HPHONE_D', 'RFA_2R', 'RFA_2F', 'RFA_2A',
       'MDMAUD_R', 'MDMAUD_F', 'MDMAUD_A', 'GEOCODE2'],
      dtype='object', length=476)

In [9]:
# Splitting feature names into groups
non_metric_features = pva_data.select_dtypes(include=['object']).columns
metric_features = pva_data.select_dtypes(exclude=['object']).columns

In [10]:
# Confirming that no variable was left behind
len(metric_features) + len(non_metric_features)

476

In [11]:
# Finding the percentage of NaN's in each column
percent_missing = pva_data.isnull().sum() * 100 / len(pva_data)
missing_value = pd.DataFrame({'Column Name': pva_data.columns,'Percentage Missing': percent_missing})
missing_value.sort_values('Percentage Missing', inplace=True)
# Printing the top 10 columns with biggest NaN values percentage
print(missing_value.tail(10))

         Column Name  Percentage Missing
RAMNT_15    RAMNT_15           92.388798
RDATE_15    RDATE_15           92.388798
RAMNT_6      RAMNT_6           99.186685
RDATE_6      RDATE_6           99.186685
RAMNT_4      RAMNT_4           99.705488
RDATE_4      RDATE_4           99.705488
RDATE_3      RDATE_3           99.746363
RAMNT_3      RAMNT_3           99.746363
RDATE_5      RDATE_5           99.990567
RAMNT_5      RAMNT_5           99.990567


In [12]:
pva_data.corr()

,Unnamed: 0,TCODE,NUMCHLD,INCOME,WEALTH1,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,...,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F
Unnamed: 0,1.000000,0.001065,0.006030,0.003137,-0.002498,0.003794,0.003216,0.005086,0.002884,0.004738,...,-0.006006,-0.005101,0.000675,0.001198,0.004318,0.009894,0.002990,0.002457,-0.004805,-0.003229
TCODE,0.001065,1.000000,0.003457,-0.005683,-0.006437,-0.002897,-0.001160,-0.006312,-0.004113,-0.003759,...,0.027063,0.026508,-0.012115,0.001127,-0.000010,0.004964,-0.005778,-0.001920,-0.002025,0.007835
NUMCHLD,0.006030,0.003457,1.000000,0.022066,0.025711,0.003927,0.028935,-0.007479,-0.006664,-0.008620,...,-0.023929,-0.021063,-0.018695,-0.010425,-0.002989,-0.011415,-0.018747,0.004462,0.024972,0.019857
INCOME,0.003137,-0.005683,0.022066,1.000000,0.456094,0.004738,-0.007168,-0.035434,0.007814,0.018775,...,-0.124218,-0.106561,0.096302,0.072044,0.091475,0.025956,0.114931,0.066211,-0.029094,-0.079972
WEALTH1,-0.002498,-0.006437,0.025711,0.456094,1.000000,0.020244,-0.003759,-0.024914,0.008433,0.040931,...,-0.083288,-0.065309,0.072440,0.055056,0.067108,0.017545,0.086482,-0.027074,-0.017367,-0.052808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TIMELAG,0.009894,0.004964,-0.011415,0.025956,0.017545,0.008685,0.001517,0.002419,0.006022,0.004416,...,-0.110699,-0.101840,0.073466,0.032012,0.073174,1.000000,0.077111,0.019095,0.008872,-0.132972
AVGGIFT,0.002990,-0.005778,-0.018747,0.114931,0.086482,-0.014705,-0.016241,-0.027719,0.015487,0.026210,...,-0.256992,-0.268607,0.754563,0.766810,0.784187,0.077111,1.000000,0.011507,-0.044323,-0.277168
CONTROLN,0.002457,-0.001920,0.004462,0.066211,-0.027074,0.000420,-0.001785,-0.017925,0.005684,0.003236,...,-0.129263,-0.081800,0.040790,-0.011947,-0.000427,0.019095,0.011507,1.000000,-0.108747,-0.016545
HPHONE_D,-0.004805,-0.002025,0.024972,-0.029094,-0.017367,0.213498,0.018020,0.013237,0.026608,0.003245,...,0.036087,0.048156,-0.048413,-0.023070,-0.029196,0.008872,-0.044323,-0.108747,1.000000,0.007607


In [13]:
# Creating a copy to apply...
pva_copy = pva_data.copy()

In [14]:
def replaceX(a_column):
    pva_copy[a_column] = pva_copy[a_column].str.replace('X','1')
    pva_copy[a_column] = pva_copy[a_column].str.replace(' ','0')

In [15]:
replaceX('NOEXCH')
replaceX('RECINHSE')
replaceX('RECP3')
replaceX('RECPGVG')
replaceX('RECSWEEP')
replaceX('MAJOR')
replaceX('PEPSTRFL')

In [16]:
#pd.set_option('display.max_columns', None)

In [17]:
pva_copy[metric_features]

,Unnamed: 0,TCODE,NUMCHLD,INCOME,WEALTH1,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,...,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F
0,0,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,...,31,14,5.0,12.0,10.0,4.0,7.741935,95515,0,4
1,1,1,1.0,6.0,9.0,16,0.0,0.0,3.0,1.0,...,3,1,10.0,25.0,25.0,18.0,15.666667,148535,0,2
2,2,1,NaN,3.0,1.0,2,0.0,0.0,1.0,0.0,...,27,14,2.0,16.0,5.0,12.0,7.481481,15078,1,4
3,3,0,NaN,1.0,4.0,2,0.0,0.0,0.0,0.0,...,16,7,2.0,11.0,10.0,9.0,6.812500,172556,1,4
4,4,0,1.0,3.0,2.0,60,1.0,0.0,9.0,0.0,...,37,8,3.0,15.0,15.0,14.0,6.864865,7112,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,95407,1,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,...,1,0,25.0,25.0,25.0,NaN,25.000000,184568,0,1
95408,95408,1,1.0,7.0,9.0,1,0.0,0.0,1.0,0.0,...,1,0,20.0,20.0,20.0,NaN,20.000000,122706,1,1
95409,95409,1,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,...,7,4,3.0,10.0,10.0,3.0,8.285714,189641,1,3
95410,95410,0,NaN,7.0,NaN,0,NaN,NaN,NaN,NaN,...,41,18,5.0,21.0,18.0,4.0,12.146341,4693,1,4


In [18]:
pva_copy['NUMCHLD'].fillna(0, inplace=True)
pva_copy.fillna(pva_copy.median(), inplace=True)
pva_copy[metric_features]

,Unnamed: 0,TCODE,NUMCHLD,INCOME,WEALTH1,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,...,NGIFTALL,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F
0,0,0,0.0,4.0,6.0,0,0.0,0.0,0.0,0.0,...,31,14,5.0,12.0,10.0,4.0,7.741935,95515,0,4
1,1,1,1.0,6.0,9.0,16,0.0,0.0,3.0,1.0,...,3,1,10.0,25.0,25.0,18.0,15.666667,148535,0,2
2,2,1,0.0,3.0,1.0,2,0.0,0.0,1.0,0.0,...,27,14,2.0,16.0,5.0,12.0,7.481481,15078,1,4
3,3,0,0.0,1.0,4.0,2,0.0,0.0,0.0,0.0,...,16,7,2.0,11.0,10.0,9.0,6.812500,172556,1,4
4,4,0,1.0,3.0,2.0,60,1.0,0.0,9.0,0.0,...,37,8,3.0,15.0,15.0,14.0,6.864865,7112,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,95407,1,0.0,4.0,6.0,0,0.0,0.0,0.0,0.0,...,1,0,25.0,25.0,25.0,6.0,25.000000,184568,0,1
95408,95408,1,1.0,7.0,9.0,1,0.0,0.0,1.0,0.0,...,1,0,20.0,20.0,20.0,6.0,20.000000,122706,1,1
95409,95409,1,0.0,4.0,6.0,0,0.0,0.0,0.0,0.0,...,7,4,3.0,10.0,10.0,3.0,8.285714,189641,1,3
95410,95410,0,0.0,7.0,6.0,0,0.0,0.0,0.0,0.0,...,41,18,5.0,21.0,18.0,4.0,12.146341,4693,1,4


# Data exploration
## -Pandas profiling 

In [ ]:
profile = ProfileReport(
    pva_copy.sample(frac=.02),
    minimal = True
    title='Tugas Customer Data',
    correlations={
        "pearson": {"calculate": True},
        "spearman": {"calculate": False},
        "kendall": {"calculate": False},
        "phi_k": {"calculate": False},
        "cramers": {"calculate": False},
    },
)
prof.to_file(output_file='output.html')

prof = ProfileReport(pva_data.sample(frac=.02), title='Tugas Customer Data', 
        correlations={
        "pearson": {"calculate": True},
        "spearman": {"calculate": False},
        "kendall": {"calculate": False},
        "phi_k": {"calculate": False},
        "cramers": {"calculate": False},
    },
)
prof.to_file(output_file='output.html')